In [ ]:
using Oscar # start up Oscar

# Worksheet 3: parametrizations and normalization

## Ideals defining images of maps

Let $f \colon X \to Y$ be an algebraic/complex analytic map. 
How can we compute the Zariski-closure of the image of $f$? 
Finding equations for the image by hand is, in many cases, almost impossible. 
Let us discuss how this can be achieved with the help of computers.

### Manual computations using eleminitation orderings

The following ideal models the graph of the map 

$f \colon \mathbb C \to \mathbb C^3, \quad t \mapsto (x,y,z) = (t^4, t^5, t^6)$.

This is a *finite* map, so its image is a closed variety. We choose an *elemination ordering* for $t$ to compute the ideal $J$ defining the image.

In [ ]:
R, (t,x,y,z) = QQ["t","x", "y", "z"]
I = ideal(R, [x-t^4, y-t^5, z-t^6])
o = degrevlex([t])*degrevlex([x,y,z])
a = std_basis(I, o)
lead_a = (x->leading_monomial(x, o)).(a) # collect the leading monomials of all generators of the standard basis


As discussed in the lectures, the elements $g$ of a standard basis whose 
leading term does not involve $t$ provide us with a set of generators for the ideal defining the image of $f$. 

In [ ]:
@show lead_a
gensJ = [g for g in a if !divides(leading_monomial(g, o), t)[1]] # select those leading monomials which are not divisible by t

### A taste of OSCAR: direct kernel computations
Compare the above procedure to the possibility to simply define the associated ring homomorphism of polynomial 
rings and its kernel: Isn't it more convenient?

In [ ]:
R, (x,y,z) = QQ["x", "y", "z"]
# S, t = QQ["t"] # DO NOT USE!!! This sets up S as a univariate 
                 # polynomial ring which is another data structure
                 # There is no method for the computation of the 
                 # kernel in this case!
S, (t,) = PolynomialRing(QQ, ["t"])
phi = hom(R, S, [t^4, t^5, t^6])
K = kernel(phi)

## Parametrization of curve singularities

### An example for manual normalization

Consider the cuspidal curve $(X,0) \subset (\mathbb C^2,0)$ defined by the equation $f$ below.

In [ ]:
R, (x,y) = QQ["x", "y"]
f = x^2 - y^3
I = ideal(R, f)

We quickly compute the singular locus. No surprises.

In [ ]:
J = radical(singular_slocus(I)) # an ideal defining the singular locus

The *coordinate ring* $A$ of $X$ is given by the quotient ring modulo $I$. 

In [ ]:
A, _ = quo(R, I)

Let us check whether we find a suitable non-zerodivisor on $A$ in $J$.

In [ ]:
zero_ideal_in_A = ideal(A, [A(0)])
K = ideal(A, [A(x)]) # x is an element in J
quotient(zero_ideal_in_A, K) # should return the zero ideal again!

Alternatively, we could work over the ring $R$ as follows:

In [ ]:
issubset(quotient(I, ideal(R, x)), I)

Back to working over the quotient ring $A$. We can pass to the ideal of $A$ generated by $J$ and also set up the ideal generated by $x \cdot J$. To shorten notation, we will in the following just speak of $J \subset A$ whenever there is no risk of confusion. 

In [ ]:
JmodI = ideal(A, A.(gens(J)))
xJmodI = ideal(A, A.(x.*gens(J))) # f.(vec) applies the function f 
                                  # to each entry of a vector 
                                  # (broadcasting)

Recall that to any $\varphi \in \mathrm{Hom}_A(J, J)$ we 
can assign a fraction $\frac{\varphi(x)}{x}$. Using 
the $A$-linearity of $\varphi$, it is easy to see that this 
is independent of the choice of the non-zerodivisor $x\in J$ on $A$. 

Then, according to the lectures, we need the quotient of these two ideals:

In [ ]:
H = quotient(xJmodI, JmodI)

We see that, besides the standard generator $\frac{x}{x}=1$, there is 
one other generator $\frac{y^2}{x}$ in the automorphism ring 
$\mathrm{End}_A(J)$ of the ideal $J = \langle x, y \rangle\subset A$. 

Indeed, we see that for both generators of $J$, $x \cdot \frac{y^2}{x} \in J$ and $y \cdot \frac{y^2}{x} \equiv \frac{x^2}{x} = x \in J$ 
modulo $I$ and hence, multiplication with $\frac{y^2}{x}$ gives a well defined endomorphism of $J$. 

If we were to use this as a step in the normalization algorithm, we would introduce an extra variable $t_1$, subject to the relation 
$x \cdot t_1 = y^2$. Moreover, as described in the lecture, we need to add further relations 
for the products of the new variables, in case there are many, and for their syzygies. We leave this as an exercise.

### Normalization using OSCAR's built-in routines

Let us compute a normalization of $A$ directly; see the OSCAR documentation for details on the output.

In [ ]:
out = normalization(A)
(Aext, normalization_map, (a, K)) = out[1] # B is the quotient ring describing the first branch of the normalization.

In [ ]:
Aext
modulus(Aext)

In [ ]:
normalization_map

The first element of the returned tuple is the quotient ring for the normalization. Instead of manually choosing an appropriate elimination ordering, we can make use of a heuristic that has been implemented in Singular (and wrapped for OSCAR): `elimpart`.

In [ ]:
elimpart(modulus(Aext)) # This describes the parametrization of the branch for us

We repeat the above with another curve which has more than one branch.

In [ ]:
I2 = ideal(R, x^2 - y^4)
A2, _ = quo(R, I2)
out = normalization(A2)
(B, phi, (a, K)) = out[1]
elimpart(modulus(B))

### One more example of normalization

The following example illustrates that one has to be careful when using primary decomposition. The ideal $I_3$ defines a space curve $(X,0) \subset (\mathbb C^3, 0)$ that seems to decompose into two components. 
But do we really only have two branches? 

In [ ]:
R, (x,y,z) = QQ["x", "y", "z"]
I3 = ideal(R, [x^4-y*z^2,x*y-z^3,y^2-x^3*z])
@show pdec = primary_decomposition(I3)
[dim(a[1]) for a in pdec] # How many components do we have?

Let us check the number of branches by computing a normalization.

In [ ]:
A, _ = quo(R, I3)
out = normalization(A)

So we only get one quotient ring describing our normalization. But nevertheless the variety might have several components. We get more insight from `elimpart`.

In [ ]:
B = out[1][1]
Rsubst, Isubst, pb = elimpart(modulus(B))

What do we see from this output? In this case, we do not only have 
two branches, but five! They are distinguished by $T_4$ which takes 
the fifth roots of unity as constant values on each branch, respectively. The parameter for the branches is $z$.

## The $\delta$-invariant 

As an example, we consider the space curve singularity 
parametrized by $t \mapsto (t^4, t^5, t^6)$. 

In [ ]:
R, (x,y,z) = QQ["x", "y", "z"]
# S, t = QQ["t"] # DO NOT USE!!! This sets up S as a univariate 
                 # polynomial ring which is another data structure
                 # There is no method for the computation of the 
                 # kernel in this case!
S, (t,) = PolynomialRing(QQ, ["t"])
phi = hom(R, S, [t^4, t^5, t^6])
K = kernel(phi)
delta_invariant(K)

Now an example of a non-complete intersection for $t \mapsto (t^4, t^5, t^7)$. 

In [ ]:
R, (x,y,z) = QQ["x", "y", "z"]
# S, t = QQ["t"] # DO NOT USE!!! This sets up S as a univariate 
                 # polynomial ring which is another data structure
                 # There is no method for the computation of the 
                 # kernel in this case!
S, (t,) = PolynomialRing(QQ, ["t"])
phi = hom(R, S, [t^4, t^5, t^7])
K = kernel(phi)
delta_invariant(K)

## Normalization of higher dimensional varieties

The following is an **I**solated **C**ohen-**M**acaulay **C**odimension $2$ surface singularity. Spoiler: These are known to be normal.

In [ ]:
R, (x,y,z,w) = QQ["x", "y", "z", "w"]
M = R[x y z; y z w]
I = ideal(R, minors(M, 2))
A, _ = quo(R, I)
out = normalization(A)
B = out[1][1]

We see that, indeed, normalization does nothing: No extra variables, no changes in the equations, nothing. Nevertheless, the space is singular:

In [ ]:
radical(singular_slocus(I))